# ParaView Batch example notebook

In this notebook we will explore running ParaView in batch mode on Cooley.


Let's assume that we have a time series of data files, and we want to generate an animation of that time series. 
* We would first run ParaView in interactive mode and set up our visualization pipeline, adding desired filters, etc.
* Once our pipeline is all set we can save the state as a Python script
 * File->Save State -> filename.py

There's an example state file named: **blood_flow_state_01.py**

Let's take a look at that first...

------



Now, we can run ParaView in batch mode (pvbatch), and pass it that Python script to render a set of frames from that time series.

Our test data set is pretty small, but you can imagine wanting to do this for a much larger data set.  In which case, it would be useful to be able to have multiple jobs processing different sets of frames at the same time, in order to reduce the overall wait time.

So, we will submit multiple jobs to run  pvbatch and render different sets of frames.

Once all of the frames are complete, we can encode these frames into a video using ffmpeg.

Finally, we will show the resulting video and embed it in this notebook

Paths and filenames

Create a directory on your home directory for frames and videos and point *framesDir* variable to it

We also specify the path to our ParaView Python state file

In [9]:
my_login="hyunlim"

framesDir="/home/" + my_login + "/atpesc/viz/jupyter-ATPESC2018/pv_blood_flow/frames/"
videoFileName="video.mp4"

pv_script="/home/" + my_login + "/atpesc/viz/jupyter-ATPESC2018/pv_blood_flow/blood_flow_state_01.py"

Our example data set has 100 time steps.  So we'll set the total number of frames. In a minute we're going to submit two jobs, each will render half of the time steps.  So, we'll also set the number of half of the frames.

In [10]:
start_frame = 0
num_frames = 100
num_frames_half = int(num_frames/2)
print ("num_frames_half: " , num_frames_half)

num_frames_half:  50


Clean up directory containing frames. 

Think carefully before running. You may not want to delete frames from a previous run


In [11]:
! [ -e "$framesDir/frame_"0000.png ] && rm "$framesDir/frame_"*png && echo "previous frames deleted" || echo "frames directory was empty"

frames directory was empty


Check to make sure frames directory is empty

In [12]:
! ls -la "$framesDir"

total 2
drwxr-xr-x 2 hyunlim users  512 Aug  9 20:30 .
drwxr-xr-x 4 hyunlim users 1024 Aug  9 21:54 ..
-rw-r--r-- 1 hyunlim users    0 Aug  9 20:30 .empty


Since we're going to be submitting jobs to the queue on Cooley, we'll need to tell it what project to charge and what queue to use.  We'll also set our login name, so that we can check on our jobs in the queue once we submit them.

During our reservation these values should be set to:

* my_project = **"ATPESC2018"**
* queue = **"training"**

In [14]:
my_project="ATPESC2018"
queue="training"

Next we will create a batch script, passing it all of the relevant information.  That script will submit two jobs, one to render the first half of our frames, and another to render the second half.

In [15]:
%%bash -s  "$my_project" "$queue" "$pv_script" "$start_frame" "$num_frames_half" "$framesDir" 
echo $1 $2 $3 $4 $5 $6
/bin/qsub -n 1 -t 30 -A $1 -q $2 --env DISPLAY=:0.0 /soft/visualization/paraview/v5.5.2/bin/pvbatch $3 $4 $5 $6
/bin/qsub -n 1 -t 30 -A $1 -q $2 --env DISPLAY=:0.0 /soft/visualization/paraview/v5.5.2/bin/pvbatch $3 $5 $5 $6



ATPESC2018 training /home/hyunlim/atpesc/viz/jupyter-ATPESC2018/pv_blood_flow/blood_flow_state_01.py 0 50 /home/hyunlim/atpesc/viz/jupyter-ATPESC2018/pv_blood_flow/frames/
1611097
1611098


Once we've run the above script, we can check the queue to see the state of our jobs:

In [16]:
! /bin/qstat -fu $my_login

JobID    JobName  User     Score    WallTime  QueuedTime  RunTime   Nodes  State     Location      Mode    Procs  Queue     StartTime                             
1611097  N/A      hyunlim    0.5    00:30:00  00:00:42    00:00:33  1      exiting   cc102.cooley  script  1      training  Thu Aug 09 21:59:10 2018 +0000 (UTC)  
1611098  N/A      hyunlim    0.7    00:30:00  00:01:12    00:00:02  1      starting  cc063.cooley  script  1      training  Thu Aug 09 21:59:41 2018 +0000 (UTC)  


We can also check our frames directory to see how many of the frames have completed.  Once this reaches 100, we will be finished rendering.  If you then go back to the previous cell, and check the queue again, you should see your jobs exiting (or nothing if they have already finished exiting).
    

In [18]:
! ls "$framesDir" | wc -l 

100


If there is an older version of your video file, delete.

Again, think twice before running the next cell. You may not want to delete this file before copying it somewhere else

In [ ]:
! [ -e "$videoFileName" ] && rm "$videoFileName" && echo "previous video file deleted" || echo "no previous video file"

We will generate a video file now with ffmpeg. Let's use the frames generated above

In [19]:
! /soft/visualization/ffmpeg/ffmpeg -r 15 -i "$framesDir"/frame_%04d.png -pix_fmt yuv420p -r 25 "$videoFileName"

ffmpeg version N-90676-g876f9ac Copyright (c) 2000-2018 the FFmpeg developers
  built with gcc 4.8.1 (GCC)
  configuration: --prefix=/projects/visualization/srizzi/ffmpeg/ffmpeg_build/ --pkg-config-flags=--static --extra-cflags=-I/projects/visualization/srizzi/ffmpeg/ffmpeg_build/include --extra-ldflags=-L/projects/visualization/srizzi/ffmpeg/ffmpeg_build/lib --extra-libs=-lpthread --extra-libs=-lm --bindir=/projects/visualization/srizzi/ffmpeg/ffmpeg_bin/ --enable-gpl --enable-libfreetype --enable-libx264 --enable-nonfree
  libavutil      56. 13.100 / 56. 13.100
  libavcodec     58. 17.100 / 58. 17.100
  libavformat    58. 11.101 / 58. 11.101
  libavdevice    58.  2.100 / 58.  2.100
  libavfilter     7. 15.100 /  7. 15.100
  libswscale      5.  0.102 /  5.  0.102
  libswresample   3.  0.101 /  3.  0.101
  libpostproc    55.  0.100 / 55.  0.100
Input #0, image2, from '/home/hyunlim/atpesc/viz/jupyter-ATPESC2018/pv_blood_flow/frames//frame_%04d.png':
  Duration: 00:00:04.00, start: 0.00

Now we embed the video in HTML format so we can view it here! if all works 
we should see a video of the simulation.

In [20]:
# import modules
import io
import os
from IPython.display import HTML
# Embed the video using HTML
HTML(data='<video controls> <source src="' + videoFileName + '" type="video/mp4"> </video>') 

# Then I am done!!!